In [3]:
import numpy
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.cuda as cuda
import torch

In [4]:
cuda.is_available()

True

In [15]:
class ResNeXt(nn.Module):    
    
    def __init__(self):
        super(ResNeXt, self).__init__()
        
        self.conv1 = self.common_layers(nn.Conv2d(in_channels=x.size()[2],
                      out_channels=64,
                      kernel_size=7,
                      strides=2,
                      padding="SAME"
                     ))
        
        slef.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        
        
    
    def common_layers(self, y):
        y = nn.BatchNorm2d(y)
        y = nn.LeakyReLU(y)
        return y

    def num_of_flat_features(self, x):
        size = x.size()[1:]
        
        num_features = 1
        
        for i in size:
            num_features *= i
        return num_features

    def group_conv(self, y, n_channels, strides):

        in_channels = y.size()[1]
        if cardinality ==1:
            return nn.Conv2d(in_channels=in_channels, 
                             out_channels=n_channels, 
                             stride=strides, 
                             kernel_size=(3,3), 
                             padding="SAME",
                             bias=True
                            )

        # Cardinality should divide n_channel in order for n_channel(s) to be spilt to #cardinality groups
        assert not n_channels % cardinality 

        # No. of layers in each split
        d = n_channels // cardinality

        # Splitting layers into different groups and convolving. 
        # Principle idea of ResNeXt
        groups = nn.Conv2d(in_channels=d, 
                           out_channels=n_channels, 
                           kernel_size=(3,3), 
                           stride=strides, 
                           groups=cardinality, 
                           bias=True, 
                           padding='SAME'
                          )

        # Concatinating the groups into a single tensor
        y = torch.cat(groups(y))

        return y


    def residual_block(self, y, n_channels_in, n_channels_out, strides=(1,1), project_shortcut=False):
        """
        The network is made of multiple residual blocks of similar structure.
        There are many skip connections between these blocks to ease training.
        """

        shortcut = y

        # WHY???
        y = nn.Conv2d(in_channels=y.size()[1],
                      out_channels=n_channels_in, 
                      kernel_size=1, 
                      stride=1, 
                      padding='SAME'
                     )(y)

        y = common_layers(y)

        # ResNeXt
        y = group_conv(y, n_channels_in, strides=strides)
        y = common_layers(y)


        y = nn.Conv2d(in_channels=y.size()[1],
                          out_channels=n_channels_out, 
                          kernel_size=1, 
                          stride=1, 
                          padding='SAME'
                         )(y)
        # applying batch norm before adding with the shortcut
        y = nn.BatchNorm2d()(y)

        # if shortcut is between layers with with differnet sizes
        if project_shortcut or strides!=(1,1):

            shortcut = nn.Conv2d(in_channels=shortcut.size()[1],
                                     out_channels=n_channels_out, 
                                     kernel_size=1, 
                                     stride=1, 
                                     padding='SAME'
                                    )(shortcut)

            shortcut = nn.BatchNorm2d(shortcut)

        y = torch.add(y, shortcut)

        y = nn.LeakyReLU(y)

        return y


    def forward(self, x, cardinality=1):
        """
        If cardinality=1, then ResNet otherwise know as ResNeXt
        """    
    
    
        
        # conv1
        x = nn.Conv2d(in_channels=x.size()[2],
                      out_channels=64,
                      kernel_size=7,
                      strides=2,
                      padding="SAME"
                     )(x)
        x = common_layers(x)

        # .pool
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        
        # ResNeXt Simplified for the purpose of training
        for i in range(3):
            project_shortcut = True if i == 0 else False
            x = residual_block(x, 128, 256, project_shortcut=project_shortcut)
        
        features = self.num_of_flat_features(x)
        x = x.view(-1, features)
        x = F.relu(nn.Linear(features, 120)(x))
        x = F.relu(nn.Linear(120, 84)(x))
        x = nn.Linear(84, 10)(x)
        
        return x
        
        
    
    

In [16]:
net = ResNeXt()
print(net)

ResNeXt(
)
